# GUE Gas Planning Calculations

This notebook provides a comprehensive set of functions and tables for gas planning based on Global Underwater Explorers (GUE) standards.

## Features
source
# This cell contains background notes (Ideal gas laws, SCR formulas, etc.).
# Interactive test invocations that relied on legacy signatures have been
# removed to allow automated execution. Run tests interactively in the
# notebook if you need to exercise those helper functions.


## Forms to calculate the various GUE calculations and put out the appropriate tables

In [29]:
import sys,os,string,time,random,math
from os import listdir
from os.path import isfile, join
import subprocess

import numpy as np
import pandas as pd

from PIL import Image
from matplotlib import pyplot as plot
from IPython.display import display, Image
%matplotlib inline

In [30]:
print(math.sqrt(2*(2.5**2)))

3.5355339059327378


### Min Gas via CAT
C: Consumption (GUE standard for one diver is .75, for min gas, we are calc for two divers:cubic ft/min)

A: Atmosphere (Depth/33)+1 is Current ATA , (Current ATA+1) / 2 is average ATA

T: Time (time in min to surface. for GUE: depth/10, plus 1 min to deal with any issues)

MG: Min Gas to reach the surface for two divers

C x A x T = MG   (result is in cubic ft gas)

In [31]:
# Use the shared helpers from gue_calc_lib to avoid redefining them in the notebook
from gue_calc_lib import (
    tanks as module_tanks, calcpTot, calcpGas, calcTimeToStop, calcTimeToSurface,
    calcATA, calcPPO2, calcMG, calcTF, calcPSI, calcCF, calcSCR, calcGasVolCons,
    calcUG, calcBottomTime, O2_PSI_to_add, trimix_P_He, trimix_PO2, nitrox_p, nitrox_FO2, trimixPP
)

# expose tanks under the notebook name for backwards compatibility
tanks = module_tanks

# quick smoke checks (silent prints can be enabled during interactive runs)
print('module tanks keys:', list(tanks.keys())[:5])
print('calcMG(100) =>', calcMG(100))

module tanks keys: ['AL80', '2xAL80', 'AL40', '2xLP85', '2xHP100']
calcMG(100) => 41


In [32]:
#mixing gases
#nitrox blending: Partial Pressure
# Target_FO2=final fraction of O2 in final mix, eg. 32%=.32 -- p=final working pressure, eg. 3000PSI

def O2_PSI_to_add(target_FractionO2=.32, p=3000):
    """Compute PSI of O2 to add to an empty tank for a given target FO2.

    Uses the partial-pressure blending approach assuming the remainder will be
    filled with air after O2 addition.

    Args:
        target_FractionO2 (float): Target fraction of O2 (e.g., 0.32 for 32%).
        p (float): Final tank pressure in PSI.

    Returns:
        float: PSI of O2 to add before topping off with air.
    """
    return ((target_FractionO2 - .21) / .79) * p    # Give amount of O2 to add to an empty tank
# fill to p with air
print('test O2PSI:(expect: 417.7)', O2_PSI_to_add())


#TriMix blending:
# 1.   F_He=Fraction Helium in final mix(eg:40%), p= final fill pressure of tank(eg: 3000psi)
def trimix_P_He(F_He=.40,p=3000): #Helium to add to tank
    """Calculate PSI of helium to add for a desired trimix composition.

    Args:
        F_He (float): Fractional helium in final mix (e.g., 0.40).
        p (float): Final fill pressure in PSI.

    Returns:
        float: PSI of helium to add.
    """
    return F_He * p    
print('test F_He:(expect: 1200)', trimix_P_He())
# 2.   F_O2 = fraction O2 in final mix
def trimix_PO2 (F_O2=.16, p=3000): #O2 final PSI
    """Calculate PSI of oxygen portion for a trimix final fraction.

    Args:
        F_O2 (float): Fractional oxygen in final mix (e.g., 0.16).
        p (float): Final fill pressure in PSI.

    Returns:
        float: PSI representing the oxygen portion of the final mix.
    """
    return F_O2 * p    
print('test PO2:(expect: 480)', trimix_PO2())
# 3. 
def nitrox_p(trimix_P_He=trimix_P_He(), p=3000):
    """Remaining PSI available for nitrox portion after adding helium.

    Args:
        trimix_P_He (float): PSI of helium added to the tank.
        p (float): Final tank pressure.

    Returns:
        float: PSI left for O2 and air (nitrox portion).
    """
    return p - trimix_P_He
print('test nitrox_p:(expect: 1800)', nitrox_p(trimix_P_He()))

# 4. 
def nitrox_FO2(trimix_PO2=trimix_PO2(),nitrox_p=nitrox_p()):
    """Compute FO2 of the nitrox remainder portion after helium removal.

    Args:
        trimix_PO2 (float): PSI of the oxygen portion for trimix.
        nitrox_p (float): PSI of the nitrox portion (after He removed).

    Returns:
        float: Fractional FO2 of the nitrox portion.
    """
    return trimix_PO2 / nitrox_p
print('test nitrox_FO2:(expect: .266)', nitrox_FO2())
# 5. O2PSI_to_add with target_FractionO2 = nitrox_FO2
print ('test final_mix:(expect:127.44)',O2_PSI_to_add(nitrox_FO2(), nitrox_p()) ) # add this much O2 to tank.

def trimixPP(F_O2=.16,F_He=.40, p=3000):
    """Print recommended partial-pressure steps to create a trimix.

    Args:
        F_O2 (float): Target oxygen fraction.
        F_He (float): Target helium fraction.
        p (float): Final fill pressure in PSI.

    Returns:
        None: Prints the steps and values for He and O2 additions.
    """
    P_He_to_add=trimix_P_He(F_He,p)
    t_po2=trimix_PO2(F_O2, p)
    nit_p=nitrox_p(P_He_to_add,p)
    nit_Fo2=nitrox_FO2(t_po2,nit_p)
    print('for trimix:',int(F_O2*100),'/',int(F_He*100))
    print('add He:',P_He_to_add)
    print('add O2:',O2_PSI_to_add(nit_Fo2, nit_p))
    print('fill with air')
print('test', trimixPP())
print('\nrun', trimixPP(F_O2=.14))


test O2PSI:(expect: 417.7) 417.7215189873418
test F_He:(expect: 1200) 1200.0
test PO2:(expect: 480) 480.0
test nitrox_p:(expect: 1800) 1800.0
test nitrox_FO2:(expect: .266) 0.26666666666666666
test final_mix:(expect:127.44) 129.1139240506329
for trimix: 16 / 40
add He: 1200.0
add O2: 129.1139240506329
fill with air
test None
for trimix: 14 / 40
add He: 1200.0
add O2: 53.1645569620254
fill with air

run None


In [33]:
# Other equations (notes only) - removed interactive test calls for automated runs
# This cell contains background notes (Ideal gas laws, SCR formulas, etc.).
# Interactive test invocations that relied on legacy signatures have been
# removed to allow automated execution. Run tests interactively in the
# notebook if you need to exercise those helper functions.


In [34]:
# scratch area (interactive tests removed for automated execution)
# Uncomment the following lines and run interactively if you want to
# exercise calcCF / calcSCR in an interactive session:
# print('test',calcSCR(calcCF(2.5,1700), calcATA(100), 15))
# print(calcCF(2.5, 1750))

print(calcTF(166,2640),(160/2640)*100)

6.5 6.0606060606060606


In [35]:
def min_gas_table(depth=100, gas_switch_depth=0, used_tanks=['2xAL80'], scr=1.5, verbose=False):
    """Generate a table of minimum gas requirements for depth increments.

    This function calculates minimum gas using `calcMG` for depth increments
    (10 ft increments starting at the gas switch depth up to `depth`). For
    each tank in `used_tanks` it converts the cubic-feet requirement to PSI
    and ensures PSI falls within sensible ranges.

    Args:
        depth (int): Maximum depth for table (ft).
        gas_switch_depth (int): Depth of gas switch (ft). Defaults to 0.
        used_tanks (list): List of tank keys to include.
        scr (float): Surface consumption rate used in calcMG (for consistency).
        verbose (bool): If True, enables verbose output from `calcMG`.

    Returns:
        pandas.DataFrame: DataFrame indexed by depth with min gas and tank PSI info.
    """
    if gas_switch_depth <= 0:
        MG_column = 'Time to surface(min)'
    else:
        MG_column = 'Time to next gas (min)'
    
    depth_increments = []
    for d in range(int(gas_switch_depth / 10), int(depth / 10) + 1):
        if d > 0:
            depth_increments.append(d * 10)
            
    rows = []
    for current_depth in depth_increments:
        mg = calcMG(current_depth, gas_switch_depth, scr, verbose=verbose)  # calculate min gas for this level
        
        row = {"depth": current_depth, "min gas(cf)": mg, MG_column: calcTimeToStop(current_depth, gas_switch_depth)}
        
        for curr_tank in used_tanks:
            tank_factor = calcTF(tanks[curr_tank]['rated_vol'], tanks[curr_tank]['rated_PSI'])
            curr_tank_psi = round(calcPSI(tank_factor, mg), -2)
            if curr_tank_psi <= 500:  # never go below 500 PSI
                curr_tank_psi = 500
            if curr_tank_psi > tanks[curr_tank]['rated_PSI']:
                curr_tank_psi = 0
                
            row[f"{curr_tank} (TF:{tank_factor}) PSI"] = curr_tank_psi
            
        rows.append(row)
        
    data = pd.DataFrame(rows)
    data.set_index('depth', inplace=True)
    
    # move the Time to surface col to end
    columns = [value for value in data.columns if value != MG_column]
    columns.append(MG_column)
    data = data[columns]

    return data


## Generating and Saving Minimum Gas (MG) Tables

### Overview
The following cells generate four different Minimum Gas tables for various dive scenarios. Each table is displayed with styling for readability and stored as a pandas DataFrame in the `mg_tables` dictionary for subsequent CSV export.

### Implementation Details
The table generation process has been designed to separate data creation from presentation:
- **Data Storage**: Raw DataFrames are stored in the `mg_tables` dictionary before any styling is applied
- **Display**: Styled versions (with captions) are created only for display purposes using `.style.set_caption()`
- **Silent Execution**: All table generation uses `verbose=False` to suppress calculation debug output
- **CSV Export**: A subsequent cell saves and verifies each table as individual CSV files

### Generated Tables
1. **MG to Surface Table** (`surface_mg_table.csv`)
   - Standard minimum gas to reach the surface from various depths
   - Depth range: 10-110 ft in 10 ft increments
   - Gas switch depth: 0 (direct to surface)
   
2. **Back Gas to 50% Deco Table** (`back_gas_to_50_table.csv`)
   - Minimum back gas required to ascend to a 70 ft gas switch for 50% Nitrox
   - Depth range: 80-170 ft in 10 ft increments
   - Gas switch depth: 70 ft
   
3. **Back Gas to 100% O2 Deco Table** (`back_gas_to_100_table.csv`)
   - Minimum back gas required to ascend to a 20 ft gas switch for 100% Oxygen
   - Depth range: 30-170 ft in 10 ft increments
   - Gas switch depth: 20 ft
   
4. **Deco Gas (50%) to Surface Table** (`deco_gas_table.csv`)
   - Minimum deco gas (50% Nitrox) required to surface
   - Depth range: 10-70 ft in 10 ft increments
   - Uses reduced SCR of 0.6 (single diver on deco gas)

### Notes
- All calculations use the CAT formula (Consumption × Average ATA × Time)
- Tank configurations include: AL80, 2×AL80, 2×LP85, 2×HP100, 2×HP133
- Deco gas table includes AL40 and AL80 configurations only


In [36]:
# Dictionary to hold the generated tables for later use
mg_tables = {}

# Common tanks for MG tables (only use tanks defined in gue_calc_lib.tanks)
use_tanks = ['AL80','2xAL80']

# --- MG to Surface Table ---
surface_mg_table = min_gas_table(depth=110,
                                 gas_switch_depth=0,
                                 used_tanks=use_tanks,
                                 verbose=False)
mg_tables['surface_mg_table'] = surface_mg_table
display(surface_mg_table.style.set_caption("MG to surface table"))

# --- Back Gas to 50% Deco Table ---
back_gas_to_50_table = min_gas_table(depth=170,
                                     gas_switch_depth=70,
                                     used_tanks=use_tanks,
                                     verbose=False)
mg_tables['back_gas_to_50_table'] = back_gas_to_50_table
display(back_gas_to_50_table.style.set_caption("Back gas, switch to 50% at 70ft"))

# --- Back Gas to 100% O2 Deco Table ---
back_gas_to_100_table = min_gas_table(depth=170,
                                      gas_switch_depth=20,
                                      used_tanks=use_tanks,
                                      verbose=False)
mg_tables['back_gas_to_100_table'] = back_gas_to_100_table
display(back_gas_to_100_table.style.set_caption("Back gas, switch to 100% at 20ft"))

# --- Deco Gas (50%) to Surface Table ---
deco_gas_table = min_gas_table(depth=70,
                               gas_switch_depth=0,
                               used_tanks=['AL80','AL40'],
                               scr=.6,
                               verbose=False)
mg_tables['deco_gas_table'] = deco_gas_table
display(deco_gas_table.style.set_caption("Deco gas: 50%, to surface, SCR=0.6"))

,min gas(cf),AL80 (TF:2.5) PSI,2xAL80 (TF:5.0) PSI,Time to surface(min)
depth,,,,
10,3,500,500,2
20,6,500,500,3
30,9,500,500,4
40,12,500,500,5
50,16,600,500,6
60,20,800,500,7
70,25,1000,500,8
80,30,1200,600,9
90,35,1400,700,10


,min gas(cf),AL80 (TF:2.5) PSI,2xAL80 (TF:5.0) PSI,Time to next gas (min)
depth,,,,
70,9,500,500,2
80,15,600,500,3
90,20,800,500,4
100,27,1000,500,5
110,33,1300,600,6
120,40,1600,800,7
130,48,1900,900,8
140,56,2200,1100,9
150,65,2600,1300,10


,min gas(cf),AL80 (TF:2.5) PSI,2xAL80 (TF:5.0) PSI,Time to next gas (min)
depth,,,,
20,5,500,500,2
30,8,500,500,3
40,11,500,500,4
50,15,600,500,5
60,20,800,500,6
70,25,1000,500,7
80,30,1200,600,8
90,36,1400,700,9
100,42,1600,800,10


,min gas(cf),AL80 (TF:2.5) PSI,AL40 (TF:1.5) PSI,Time to surface(min)
depth,,,,
10,1,500,500,2
20,2,500,500,3
30,3,500,500,4
40,5,500,500,5
50,6,500,500,6
60,8,500,500,7
70,10,500,600,8


In [37]:
# Save and verify the min gas tables
for name, table_df in mg_tables.items():
    filename = f"{name}.csv"
    print(f"--- Processing {name} ---")
    
    try:
        # Save the table
        table_df.to_csv(filename)
        print(f"✓ Successfully saved to {filename}")

        # Verify by reading back the saved file
        if isfile(filename):
            # Load the saved CSV to verify it's readable
            verify_df = pd.read_csv(filename, index_col=0)
            print(f"✓ Verification successful - loaded {len(verify_df)} rows, {len(verify_df.columns)} columns")
            print(f"  Columns: {', '.join(verify_df.columns.tolist())}")
            print(f"  Depth range: {verify_df.index.min()} to {verify_df.index.max()} ft")
        else:
            print(f"✗ Verification failed: {filename} not found after save.")
            
    except Exception as e:
        print(f"✗ An error occurred while processing {name}: {e}")
    print()



--- Processing surface_mg_table ---
✓ Successfully saved to surface_mg_table.csv
✓ Verification successful - loaded 11 rows, 4 columns
  Columns: min gas(cf), AL80 (TF:2.5) PSI, 2xAL80 (TF:5.0) PSI, Time to surface(min)
  Depth range: 10 to 110 ft

--- Processing back_gas_to_50_table ---
✓ Successfully saved to back_gas_to_50_table.csv
✓ Verification successful - loaded 11 rows, 4 columns
  Columns: min gas(cf), AL80 (TF:2.5) PSI, 2xAL80 (TF:5.0) PSI, Time to next gas (min)
  Depth range: 70 to 170 ft

--- Processing back_gas_to_100_table ---
✓ Successfully saved to back_gas_to_100_table.csv
✓ Verification successful - loaded 16 rows, 4 columns
  Columns: min gas(cf), AL80 (TF:2.5) PSI, 2xAL80 (TF:5.0) PSI, Time to next gas (min)
  Depth range: 20 to 170 ft

--- Processing deco_gas_table ---
✓ Successfully saved to deco_gas_table.csv
✓ Verification successful - loaded 7 rows, 4 columns
  Columns: min gas(cf), AL80 (TF:2.5) PSI, AL40 (TF:1.5) PSI, Time to surface(min)
  Depth range: 10 

In [38]:
def gas_use_table(scr=.75, time=5, depth=170, gas_switch_depth=33, used_tanks=['2xAL80'], saftey_factor=1):
    """Generate a gas usage table at varying depths for given tanks.

    Args:
        scr (float): Surface Consumption Rate (cu ft / ATA / min).
        time (int): Time window in minutes for which to show gas usage.
        depth (int): Maximum depth to include in table (ft).
        gas_switch_depth (int): Minimum depth to start table (ft).
        used_tanks (list): List of tank keys to include.
        saftey_factor (float): Multiplier to apply for safety margin.

    Returns:
        pandas.DataFrame: DataFrame indexed by depth showing ATA and gas usage per tank.
    """
    depth_increments = []
    for depth_val in range(int(gas_switch_depth / 33), int(depth / 33) + 1):
        if depth_val > 0:
            depth_increments.append(depth_val * 33)
    depths = []
    for depth_ft in depth_increments:
        gasVolCons = round(calcGasVolCons(scr, calcATA(depth_ft), time=1), 2) * saftey_factor

        curr_depth = {"depth": depth_ft, "ATA": calcATA(depth_ft), "vol gas(cf/" + str(time) + "m)": gasVolCons, }
        for curr_tank in used_tanks:  
            # generate the info about this tank at this level
            tank_factor = calcTF(tanks[curr_tank]['rated_vol'], tanks[curr_tank]['rated_PSI'])
            curr_tank_psi = round(calcPSI(tank_factor, gasVolCons))                
            # add the generated info for tank to this depth. include TF in the name for info
            curr_depth[curr_tank + " " + "(PSI/" + str(time) + "m)"] = curr_tank_psi * time
            
        # display(curr_depth)   
        depths.append(curr_depth)
        gas_data = pd.DataFrame(depths)
        gas_data.set_index('depth', inplace=True)
    return gas_data
        
    

# Example usage (displayed in notebook cells)
used_tanks = ['AL40', 'AL80', '2xAL80']

gas_data = gas_use_table(scr=.75,
                        time=5,
                        depth=170,
                        gas_switch_depth=0,
                        used_tanks=used_tanks).style.set_caption("gas_use_table:")

display(gas_data)
#  ^ make chart that says Ft3 and PSI per 1 and 5 minute at each 10 ft depth, (like gas management pg 25). 
#   uses formula above with DT=1 and 5, 


TypeError: calcGasVolCons() got an unexpected keyword argument 'time'

In [ ]:
#generate the deco gas use table
used_tanks=['AL40','AL80'] # tanks listed in the table
#gas_data= gas_table(scr=.75,time=5,depth=33, gas_switch_depth=0, used_tanks=used_tanks,saftey_factor=1.0)

display(gas_use_table(scr=.75,
                      time=5,
                      depth=33,
                      gas_switch_depth=0,
                      used_tanks=used_tanks,
                      saftey_factor=1.0).style.set_caption("gas_use_table: 100% O2, .75 scr"))

display(gas_use_table(scr=.75,
                      time=5,
                      depth=70,
                      gas_switch_depth=0,
                      used_tanks=used_tanks,
                      saftey_factor=1.0).style.set_caption('gas_use_table: 50% O2, .75 scr'))

,ATA,vol gas(cf/5m),AL40 (PSI/5m),AL80 (PSI/5m)
depth,,,,
33,2.000000,1.500000,500,0


,ATA,vol gas(cf/5m),AL40 (PSI/5m),AL80 (PSI/5m)
depth,,,,
33,2.000000,1.500000,500,0
66,3.000000,2.250000,500,0


In [ ]:
mg=calcMG(77, verbose=True)
##mg=calcMG(int(input('Depth:')))
print("min gas:",mg,'cubic feet')
ATA_70=calcATA(70)
ATA_20=calcATA(20)
avg_20_70=(ATA_70+ATA_20)/2
avg_0_20=(1+ATA_20)/2
print('70:',ATA_70,'20:',ATA_20,'avg:',avg_20_70,avg_0_20)

Consumption: 1.5 Average ATA: 2.15 time: 8
min gas: 26 cubic feet
70: 3.1 20: 1.6 avg: 2.35 1.3


In [ ]:
#T1 Gas Managment notes
#Depth Consumption Rate = DCR = SCR * ATA
#Volume of Gas = DCR * time
#Deco Gas DG_FT3 = (SCR x ATA_AVG x DecoTime(DT)) x 1.5



### GUE standard bottom gas table
| Range | Range | Gasmix | MaxpO2 | EADD | EADD | END | END | MOD | MOD|
|-------|-------|--------|--------|------|------|-----|-----|-----|----|
|0-30 m | 0-100 ft| Nx32 | 1.28   | 30 m | 100ft| 30 m|100 ft| 30 m |100 ft|
|0-30 m | 0-100 ft| 30/30| 1.20   | 20 m | 66 ft| 18 m|60 ft| 30 m| 100 ft|
|0-45 m | 0-150 ft| 21/35| 1.15   | 29 m | 97 ft| 26 m|85 ft| 45 m| 150 ft|
|0-60 m | 0-200 ft| 18/45| 1.26   | 33 m | 110ft| 29 m|95 ft| 60 m| 200 ft|

NOTE: The maximum depth for a Tech 1 diver using 18/45 is 51m/170ft despite the actual MOD of the gas mix. Equivalent air density depth: ideal limit ≤ 31 m/102 ft, USN.


#### Compendium of formulas:   
Partial pressure:  pGAS = ATA x fGAS<BR>
Maximum Operating Depth:  MOD = 33 x ((pO21.2 ÷ fO2) – 1)<BR>
Equivalent Narcotic Depth:  END = (depth + 33) x (1 – fHe) – 33<BR>
Equivalent Air Density Depth:  EADD = ((depth + 33) x (ρGAS MIX ÷ 1.29)) – 33<BR>
Central Nervous System percent: <BR>
    * CNS% = (BT + Nx50TIME) ÷ 2<BR>
         or if Nx50 is replaced with 100% O2<BR>
    * CNS% = (BT ÷ 2) + (O2TIME x 2)<BR>

#### Terminology:
pAMB Ambient pressure in ATA<BR>
pGAS Partial pressure of gas<BR>
fGAS Fraction of gas ρ <BR>      
or ρGAS MIX Density of gas or gas mix<BR>
BT Bottom time<BR>
Nx50 Nitrox 50%<BR>

### deco rule:
150ft = 1:1 bottom to deco
    +10ft add 5 m
    -10ft sub 5 m
 
for 130-180 ft

#EOF